In [23]:
import numpy as np
import csv
from math import sqrt
from scipy.special import erfc
import numpy as np
from math import erf, sqrt


# ------------------------
# Utilities
# ------------------------

VISIBLE_PDGS = np.array([
    211, -211,    # pi±
    111,          # pi0
    321, -321,    # K±
    2212, -2212,  # p, p̅
    2112, -2112,  # n, n̅
    130,          # K_L0
    22,           # γ
    11, -11,      # e±
    13, -13       # μ±
], dtype=int)

CHARGED_PDGS = np.array([
    211, -211,
    321, -321,
    2212, -2212,
    11, -11,
    13, -13
], dtype=int)

LEPTON_PDGS = {11, -11, 13, -13}

def compute_thrust_with_angle2(px, py, pz, n_iter=20, tol=1e-8):
    """
    Iterative thrust with |p|-seed, early convergence, and sign-robust stopping.

    Returns
    -------
    thrust_val : float
    cos_theta_thrust : float
    axis : np.ndarray shape (3,)
    """
    p = np.column_stack((px, py, pz)).astype(float)
    p_mag = np.linalg.norm(p, axis=1)
    denom = p_mag.sum()
    if denom == 0.0:
        return 0.0, 1.0, np.array([0.0, 0.0, 1.0])

    # Seed with direction of largest |p|
    lead = np.argmax(p_mag)
    axis = p[lead]
    norm = np.linalg.norm(axis)
    axis = axis / norm if norm > 0.0 else np.array([0.0, 0.0, 1.0])

    for _ in range(n_iter):
        # s_i = sign(p_i · n); resolve exact zeros deterministically as +1
        proj = p @ axis
        signs = np.sign(proj)
        signs[signs == 0.0] = 1.0

        new_axis = (signs[:, None] * p).sum(axis=0)
        new_norm = np.linalg.norm(new_axis)
        if new_norm == 0.0:
            break
        new_axis /= new_norm

        # Convergence up to a global sign: stop when n and new_n are aligned
        if 1.0 - abs(np.dot(axis, new_axis)) < tol:
            axis = new_axis
            break

        axis = new_axis

    thrust_val = np.sum(np.abs(p @ axis)) / denom
    cos_theta_thrust = abs(axis[2])
    return thrust_val, cos_theta_thrust, axis

def compute_thrust_event_aleph_like(px, py, pz, n_iter=10, n_random=5):
    """
    ALEPH-like thrust finder:
    - Uses |p|, |pz|, and a few random seeds
    - Less 'perfect' than all-seeds, closer to ALEPH reconstruction
    """
    p = np.column_stack((px, py, pz))
    p_mag = np.linalg.norm(p, axis=1)
    total_mag = np.sum(p_mag)

    if total_mag == 0:
        return 0.0, 1.0, np.array([0., 0., 1.])

    seeds = []

    # 1. Seed from largest |p|
    idx_maxp = np.argmax(p_mag)
    seeds.append(p[idx_maxp] / p_mag[idx_maxp])

    # 2. Seed from largest |pz|
    idx_maxpz = np.argmax(np.abs(p[:, 2]))
    if p_mag[idx_maxpz] > 0:
        seeds.append(p[idx_maxpz] / p_mag[idx_maxpz])

    # 3. Random seeds from visible particles
    n_available = len(p)
    if n_available > 0:
        n_rand = min(n_random, n_available)
        rand_idx = np.random.choice(n_available, n_rand, replace=False)
        for i in rand_idx:
            if p_mag[i] > 0:
                seeds.append(p[i] / p_mag[i])

    best_thrust = -1.0
    best_axis = np.array([0., 0., 1.])

    # Iterate thrust maximization
    for axis in seeds:
        prev_thrust = -1.0
        for _ in range(n_iter):
            dots = p @ axis
            signs = np.sign(dots)
            new_axis = np.sum(signs[:, None] * p, axis=0)
            norm = np.linalg.norm(new_axis)
            if norm == 0:
                break
            axis = new_axis / norm
            thrust_val = np.sum(np.abs(p @ axis)) / total_mag
            if abs(thrust_val - prev_thrust) < 1e-6:
                break
            prev_thrust = thrust_val

        # Final thrust
        thrust_val = np.sum(np.abs(p @ axis)) / total_mag
        if thrust_val > best_thrust:
            best_thrust = thrust_val
            best_axis = axis

    cos_theta_thrust = abs(best_axis[2])
    return best_thrust, cos_theta_thrust, best_axis

def _vt_unit(px, py, eps=1e-12):
    pT = np.hypot(px, py)
    vx = np.divide(px, pT, out=np.zeros_like(px, dtype=float), where=pT>eps)
    vy = np.divide(py, pT, out=np.zeros_like(py, dtype=float), where=pT>eps)
    return vx, vy, pT

def _sigma_d0_um(px, py, pz, a_um=25.0, b_um=95.0):
    """ALEPH-like σ(d0): sqrt(25^2 + (95/p)^2) in microns."""
    p = np.sqrt(px**2 + py**2 + pz**2) + 1e-12
    return np.sqrt(a_um**2 + (b_um/p)**2)

def _erfc_half_from_absS(S_abs):
    """
    Return 0.5*erfc(S/sqrt(2)) for array-like S_abs using
    Abramowitz–Stegun 7.1.26 (no SciPy/np.erfc needed).
    """
    S_abs = np.asarray(S_abs, float)
    y = S_abs / np.sqrt(2.0)
    # A&S constants
    p  = 0.3275911
    a1 = 0.254829592; a2 = -0.284496736; a3 = 1.421413741
    a4 = -1.453152027; a5 =  1.061405429
    t = 1.0/(1.0 + p*y)
    poly = (((((a5*t + a4)*t + a3)*t + a2)*t + a1)*t)
    erfc_y = poly * np.exp(-y*y)
    return 0.5 * erfc_y

def track_pv_probability_simple(x_mm, y_mm, px, py, pz,
                                a_um=25.0, b_um=95.0,
                                sigma_scale=1.3977865, S_cap=5.0):
    """
    PV probability using transverse d0 only.
    x,y in mm; p in GeV; σ(d0) in μm. No SciPy needed.
    """
    p = sqrt(px*px + py*py + pz*pz)
    if p <= 0.0:
        return 1.0
    pT = sqrt(px*px + py*py)
    if pT <= 0.0:
        return 1.0

    vx, vy = px/pT, py/pT
    d0_mm = abs(x_mm*vy - y_mm*vx)

    # σ(d0) in μm, with your global scale
    sig = sqrt((a_um*sigma_scale)**2 + (b_um*sigma_scale/p)**2)
    S   = (1e3 * d0_mm) / max(sig, 1e-3)   # mm → μm
    S   = min(abs(S), S_cap)               # cap |S| to avoid one crazy track dominating

    # one-sided PV prob: 0.5*erfc(S/√2) via Abramowitz–Stegun 7.1.26
    y = S / sqrt(2.0)
    pc, a1,a2,a3,a4,a5 = 0.3275911, 0.254829592, -0.284496736, 1.421413741, -1.453152027, 1.061405429
    t = 1.0/(1.0 + pc*y)
    erfc_y = (((((a5*t + a4)*t + a3)*t + a2)*t + a1)*t) * np.exp(-y*y)
    return float(0.5 * erfc_y)

def hemisphere_btag_aleph_simple(
    x_mm, y_mm, px, py, pz, pdg,
    alpha_hemi_cut=0.001, min_tracks=7, pmin=0.5,
    a_um=25.0, b_um=95.0, ip_cap_mm=4.0,
    sigma_scale=1.3977865, S_cap=5.0, use_topk=4
):
    """
    Tag a PRESELECTED hemisphere (you already masked it).
    Good tracks: |PDG| in CHARGED_PDGS and p > pmin.
    Product of the K most displaced tracks (smallest PV probs).
    """
    x = np.asarray(x_mm, float); y = np.asarray(y_mm, float)
    px = np.asarray(px, float);  py = np.asarray(py, float);  pz = np.asarray(pz, float)
    pdg = np.asarray(pdg)

    p = np.sqrt(px**2 + py**2 + pz**2)
    good = (np.isin(np.abs(pdg), CHARGED_PDGS) & (p > pmin))

    # IP sanity: drop huge-d0 tracks
    vx, vy, _ = _vt_unit(px, py)
    d0_mm = np.abs(x*vy - y*vx)
    good &= (d0_mm < ip_cap_mm)

    idx = np.flatnonzero(good)
    n_good = idx.size
    if n_good < min_tracks:
        return False, 1.0, int(n_good), []

    # per-track PV probs
    probs = np.array([
        track_pv_probability_simple(x[i], y[i], px[i], py[i], pz[i],
                                    a_um=a_um, b_um=b_um,
                                    sigma_scale=sigma_scale, S_cap=S_cap)
        for i in idx
    ], float)
    probs = np.clip(probs, 1e-300, 1.0)

    # product of K most displaced (smallest probs)
    if use_topk and probs.size > use_topk:
        kth = np.partition(probs, use_topk-1)[:use_topk]
        alpha_hemi = float(np.exp(np.sum(np.log(kth))))
    else:
        alpha_hemi = float(np.exp(np.sum(np.log(probs))))

    return (alpha_hemi < alpha_hemi_cut), alpha_hemi, int(n_good), probs.tolist()



def compute_missing_energy(px, py, pz, E, pdg, mask1, mask2,
                           sqrt_s=91.2, method="aleph", tol=1e-6):
    """
    ALEPH-style missing energy:
    - Compute visible hemisphere energies excluding neutrinos
    - Compute missing energy as E_true - E_vis.
    - Returns also the hemisphere invariant masses (m1^2, m2^2).
    
    Parameters
    ----------
    px,py,pz,E : arrays
        Particle kinematics
    pdg : array
        PDG codes
    mask1,mask2 : bool arrays
        Hemisphere assignments
    sqrt_s : float
        CM energy [GeV]
    method : {"simple","aleph"}
        Method for computing E_true
    tol : float
        Numerical tolerance for negative m^2
    """

    s = sqrt_s**2
    E_beam = sqrt_s / 2.0

    # --- Visible hemisphere energies (exclude neutrinos)
    vis_mask1 = mask1 & np.isin(pdg, list(VISIBLE_PDGS), assume_unique=True)
    vis_mask2 = mask2 & np.isin(pdg, list(VISIBLE_PDGS), assume_unique=True)

    E_vis1 = np.sum(E[vis_mask1])
    E_vis2 = np.sum(E[vis_mask2])

    # --- Invariant masses
    def m2(px, py, pz, E, mask):
        E_h  = E[mask].sum()
        px_h = px[mask].sum()
        py_h = py[mask].sum()
        pz_h = pz[mask].sum()
        val = E_h**2 - (px_h**2 + py_h**2 + pz_h**2)
        # safeguard: clip only tiny negatives
        if val < -tol:
            return val  # keep genuine negatives
        return max(val, 0.0)

    m1_sq = m2(px, py, pz, E, vis_mask1)
    m2_sq = m2(px, py, pz, E, vis_mask2)

    # --- E_true depending on method
    if method == "simple":
        E_true1 = E_beam
        E_true2 = E_beam

    elif method == "aleph":
        E_true1 = (s + m1_sq - m2_sq) / (2 * sqrt_s)
        E_true2 = (s + m2_sq - m1_sq) / (2 * sqrt_s)

    else:
        raise ValueError("method must be 'simple' or 'aleph'")

    # --- Missing energies
    E_miss1 = E_true1 - E_vis1
    E_miss2 = E_true2 - E_vis2

    return (E_vis1, E_miss1, m1_sq), (E_vis2, E_miss2, m2_sq)

In [66]:
def compute_btag_efficiency_strict(
    input_path,
    sqrt_s=91.2,
    progress_step=10_000,
    # --- b-tag parameters (JP-based working point) ---
    alpha_hemi_cut=0.001,   # JP CL cut 
    min_tracks=7,           
    pmin=0.5,
    a_um=25.0,
    b_um=95.0,
    ip_cap_mm=4.0,
    sigma_scale=1.3978,
    # --- analysis knobs ---
    thrust_min=0.85,
    cos_theta_max=0.7,
    opp_emiss_cap=20.0,
    diag_limit=50,
    NMAX=50_000
):
    """
    Diagnose b-tagging with JP CL and a topology check on the opposite hemisphere.
    - JP CL uses ALL good-track PV probabilities (uniform for light jets).
    - Extra topology: require >=2 positively-signed displaced tracks (|S|>2.5).
    - alpha_hemi_cut is used as the JP CL cut.
    Returns (efficiency_over_all_events, diagnostic_info).
    Units: positions in mm; momenta in GeV.
    """

     # --- topology requirement (heavy-flavour pattern) ---
    POS_S_MIN  = 3
    N_POS_MIN  = 2     
    POS_S_HARD = 3.7


    total_events = 0
    btagged_events = 0     # counts events passing b-tag (regardless of opp_emiss)
    selected = 0           # counts events passing b-tag AND opp_emiss hygiene
    cut_counts = {"thrust": 0, "cos": 0, "opp_emiss": 0}
    diagnostic_info = []

    with open(input_path, "r") as fin:
        for ev_id, raw in enumerate(fin):
            if ev_id >= NMAX:
                break
            line = raw.strip()
            if not line or line.startswith("#"):
                continue

            tk = line.split()
            if len(tk) % 9 != 0:
                raise ValueError(f"Line has {len(tk)} tokens (not multiple of 9).")

            arr = np.fromiter((float(x) for x in tk), dtype=float).reshape(-1, 9)
            pdg = arr[:, 0].astype(int)
            px, py, pz, E = arr[:, 1:5].T
            x, y, z, r = arr[:, 5:9].T

            total_events += 1
            if progress_step and (total_events % progress_step == 0):
                print(f"[INFO] Processed {total_events:,} events...")

            # visible subset for thrust axis
            vis_mask = np.isin(pdg, VISIBLE_PDGS, assume_unique=True)

            # --- thrust + cosθ ---
            thrust, cos_theta, axis = compute_thrust_with_angle2(px[vis_mask], py[vis_mask], pz[vis_mask])
            if thrust <= thrust_min:
                continue
            cut_counts["thrust"] += 1
            if cos_theta >= cos_theta_max:
                continue
            cut_counts["cos"] += 1

            # --- hemispheres (use axis from visible set) ---
            dots = px * axis[0] + py * axis[1] + pz * axis[2]
            mask1, mask2 = dots > 0, dots <= 0

            # --- hemisphere missing energies ---
            (E_vis1, E_miss1, _), (E_vis2, E_miss2, _) = compute_missing_energy(
                px, py, pz, E, pdg, mask1, mask2, sqrt_s=sqrt_s, method="aleph"
            )

            # signal = hemi with larger Emiss; tag the opposite
            sig_idx = 0 if E_miss1 > E_miss2 else 1
            opp_idx = 1 - sig_idx
            opp_mask = mask1 if opp_idx == 0 else mask2
            final_mask = vis_mask & opp_mask

            opp_emiss = E_miss1 if opp_idx == 0 else E_miss2
            meets_opp_emiss = (opp_emiss <= opp_emiss_cap) if (opp_emiss_cap is not None) else True
            if meets_opp_emiss:
                cut_counts["opp_emiss"] += 1

            # --- extract opposite-hemisphere visible tracks ---
            x_opp, y_opp = x[final_mask], y[final_mask]
            px_opp, py_opp, pz_opp = px[final_mask], py[final_mask], pz[final_mask]
            pdg_opp = pdg[final_mask]

            # --- per-track PV probabilities using your simple helper (no decision here) ---
            # set alpha cut wide (1.0) and no topK so we get ALL probs; enforce multiplicity via min_tracks
            is_raw, alpha_raw, n_good_btag, track_probs = hemisphere_btag_aleph_simple(
                x_mm=x_opp, y_mm=y_opp,
                px=px_opp, py=py_opp, pz=pz_opp, pdg=pdg_opp,
                alpha_hemi_cut=1.0,           # don't decide here
                min_tracks=min_tracks, pmin=pmin,
                a_um=a_um, b_um=b_um, ip_cap_mm=ip_cap_mm,
                sigma_scale=sigma_scale, S_cap=5.0,
                use_topk=0                    # use ALL good tracks
            )
            if n_good_btag < min_tracks:
                continue

            # --- Jet-Probability CL over ALL good tracks ---
            probs = np.clip(np.asarray(track_probs, float), 1e-300, 1.0)
            alpha_all = float(np.exp(np.sum(np.log(probs))))  # product of p_i
            T = -np.log(alpha_all)
            jp_sum = 1.0
            term = 1.0
            for j in range(1, n_good_btag):
                term *= T / j
                jp_sum += term
            jp_cl = alpha_all * jp_sum
            jp_pass = (jp_cl < alpha_hemi_cut)

            # --- require ≥3 positive |S|>2.7 AND at least one |S|>3.5 ---
            pT = np.hypot(px_opp, py_opp)
            vx = np.divide(px_opp, pT, out=np.zeros_like(px_opp), where=pT > 1e-12)
            vy = np.divide(py_opp, pT, out=np.zeros_like(py_opp), where=pT > 1e-12)
            d0_mm = np.abs(x_opp * vy - y_opp * vx)
            p3 = np.sqrt(px_opp**2 + py_opp**2 + pz_opp**2) + 1e-12
            sigma_um = np.sqrt((a_um * sigma_scale)**2 + (b_um * sigma_scale / p3)**2)
            S_abs = np.minimum(1e3 * d0_mm / np.maximum(sigma_um, 1e-3), 5.0)
            sgn = np.sign(x_opp * py_opp - y_opp * px_opp) * np.sign(axis[2])

            n_pos_disp = int(np.count_nonzero((sgn > 0) & (S_abs > POS_S_MIN)))
            hard_hit   = bool(np.any((sgn > 0) & (S_abs > POS_S_HARD)))
            topo_pass  = (n_pos_disp >= N_POS_MIN) and hard_hit

            is_b_tagged = (jp_pass and topo_pass)

            # --- counting ---
            if is_b_tagged:
                btagged_events += 1
                if meets_opp_emiss:
                    selected += 1

            # --- diagnostics (first diag_limit events) ---
            if len(diagnostic_info) < diag_limit:
                # avg d0 over good tracks (μm)
                vx_all, vy_all, _ = _vt_unit(px_opp, py_opp)
                d0_mm_all = np.abs(x_opp * vy_all - y_opp * vx_all)
                p_opp = np.sqrt(px_opp**2 + py_opp**2 + pz_opp**2)
                good_mask = (np.isin(np.abs(pdg_opp), CHARGED_PDGS) &
                             (p_opp > pmin) & (d0_mm_all < ip_cap_mm))
                d0_um_good = (1e3 * d0_mm_all[good_mask]) if np.any(good_mask) else np.array([])
                avg_d0_um = float(np.mean(d0_um_good)) if d0_um_good.size else 0.0
                min_prob = float(np.min(probs)) if probs.size else 1.0

                diagnostic_info.append({
                    "event_id": ev_id,
                    "n_total_tracks": int(final_mask.sum()),
                    "n_good_tracks": int(n_good_btag),
                    "alpha_all": float(alpha_all),
                    "jp_cl": float(jp_cl),
                    "avg_d0_um": float(avg_d0_um),
                    "min_track_prob": float(min_prob),
                    "n_pos_disp": n_pos_disp,
                    "is_b_tagged": bool(is_b_tagged),
                    "E_miss_opp": float(opp_emiss),
                    "meets_opp_emiss": bool(meets_opp_emiss),
                })

    # ---- print diagnostics ----
    print("\n=== DETAILED DIAGNOSTICS ===")
    print("Event | nTracks | nGood |   alpha_all |    JP_CL |  avg_d0 | min_prob | n_posS | btag |  E_miss(opp)")
    print("----------------------------------------------------------------------------------------------------")
    for d in diagnostic_info[:20]:
        print(f"{d['event_id']:5d} | {d['n_total_tracks']:7d} | {d['n_good_tracks']:5d} | "
              f"{d['alpha_all']:11.3e} | {d['jp_cl']:8.3e} | {d['avg_d0_um']:7.1f}μm | "
              f"{d['min_track_prob']:8.1e} | {d['n_pos_disp']:6d} | "
              f"{str(d['is_b_tagged']):>5} | {d['E_miss_opp']:6.1f}GeV")

    if diagnostic_info:
        n_btag_diag = sum(1 for d in diagnostic_info if d["is_b_tagged"])
        frac = 100.0 * n_btag_diag / len(diagnostic_info)
        print(f"\nDiagnostic sample: {n_btag_diag}/{len(diagnostic_info)} b-tagged ({frac:.1f}%)")

    # ---- summary ----
    eff = btagged_events / total_events if total_events > 0 else 0.0
    print("\n--- Cutflow summary ---")
    print(f"total       : {total_events:,}  (100.000000%)")
    print(f"thrust      : {cut_counts['thrust']:,}  ({100*cut_counts['thrust']/max(total_events,1):.6f}%)")
    print(f"cos         : {cut_counts['cos']:,}  ({100*cut_counts['cos']/max(total_events,1):.6f}%)")
    if opp_emiss_cap is not None:
        print(f"opp_emiss   : {cut_counts['opp_emiss']:,}  ({100*cut_counts['opp_emiss']/max(total_events,1):.6f}%)")
    print("\n--- B-tagging efficiency ---")
    print(f"Total hadronic events : {total_events:,}")
    print(f"B-tagged (opp hemi)   : {btagged_events:,}")
    print(f"Efficiency            : {eff*100:.2f}%")
    print(f"Final selected        : {selected:,}")

    return eff, diagnostic_info


In [67]:
filename = "z-decay-productsONLYb.txt"

efficiency, info = compute_btag_efficiency_strict(
    filename
)

[INFO] Processed 10,000 events...

=== DETAILED DIAGNOSTICS ===
Event | nTracks | nGood |   alpha_all |    JP_CL |  avg_d0 | min_prob | n_posS | btag |  E_miss(opp)
----------------------------------------------------------------------------------------------------
    0 |      28 |     9 |   1.219e-34 | 4.651e-24 |   472.6μm |  2.9e-07 |      9 |  True |    1.5GeV
    3 |      29 |     8 |   8.857e-18 | 3.058e-10 |   245.9μm |  2.9e-07 |      4 |  True |    0.0GeV
    5 |      35 |     9 |   4.995e-04 | 6.479e-01 |     7.1μm |  1.7e-01 |      2 | False |    0.4GeV
    6 |      25 |    10 |   1.378e-07 | 4.781e-02 |    29.2μm |  3.3e-04 |      1 | False |    0.0GeV
   10 |      31 |    10 |   4.979e-40 | 6.195e-28 |   644.1μm |  2.9e-07 |      5 |  True |    0.0GeV
   11 |      31 |    10 |   2.512e-04 | 6.802e-01 |     9.3μm |  3.0e-01 |      4 | False |    0.4GeV
   13 |      27 |     8 |   1.497e-42 | 2.459e-32 |   618.7μm |  2.9e-07 |      3 |  True |    0.0GeV
   15 |      30 |   

In [68]:
filename = "selected_hadronic_smeared.txt"

efficiency, info = compute_btag_efficiency_strict(
    filename
)

[INFO] Processed 10,000 events...
[INFO] Processed 20,000 events...
[INFO] Processed 30,000 events...
[INFO] Processed 40,000 events...
[INFO] Processed 50,000 events...

=== DETAILED DIAGNOSTICS ===
Event | nTracks | nGood |   alpha_all |    JP_CL |  avg_d0 | min_prob | n_posS | btag |  E_miss(opp)
----------------------------------------------------------------------------------------------------
    2 |      30 |     9 |   3.089e-08 | 1.065e-02 |    41.9μm |  5.6e-03 |      2 | False |   -0.3GeV
    9 |      21 |     7 |   5.140e-09 | 4.890e-04 |    49.5μm |  2.1e-06 |      4 |  True |   -4.8GeV
   15 |      26 |     9 |   1.953e-03 | 8.217e-01 |     0.0μm |  5.0e-01 |      0 | False |    1.3GeV
   22 |      17 |     7 |   4.486e-09 | 4.442e-04 |   299.3μm |  2.9e-07 |      1 | False |    0.7GeV
   27 |      30 |    11 |   1.775e-17 | 4.759e-08 |   234.2μm |  2.9e-07 |      3 |  True |   -0.5GeV
   31 |      25 |     7 |   7.812e-03 | 7.835e-01 |     0.0μm |  5.0e-01 |      2 | Fals

In [22]:
Rb = 0.216
total_inclusive = 50000
kept_inclusive = 7828
kept_bbbar = 3827
preselected_bbbar = 5422
A_tot = kept_inclusive / total_inclusive
eps_b = kept_bbbar / preselected_bbbar      # ≈ overall / preselection on b b̄
w = (A_tot - Rb*eps_b) / (1 - Rb)
print(eps_b, A_tot, w)


0.7058281077093324 0.15656 0.005231031549469662
